**Introduction**

The objective of this model will be to test which form of word reduction, stemming or lemmatization, leads to the best performing Naive Bayes classifier for movie reviews 

In [1]:
!pip install nltk 
!pip install regex
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import os
import regex as re 
import sklearn 
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download("omw-1.4")
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Dealing with unstructured data, specifically text, can get tedious because it takes many forms like reviews or articles. All these forms need to be stored as documents that then need to be compiled into a corpus. In this case each document (review) is stored in a text file and a function is needed to extract the reviews from the text file and land it into a pandas dataframe. Populating a dataset with the reviews will make the corpus of text more readable and easier to analyze. Therefore, a pandas dataframe is created with two variables:

-Sentiment: the class label of the review

-Review: textual content of movie review

The unit of analysis is the sentiment expressed by the movie review text

In [22]:
neg_path = '/workspaces/Naive-Reviews/neg/'
pos_path = '/workspaces/Naive-Reviews/pos/'

def get_docs(path, label):

    """ Extractor of text from text files stored in a directory. Populator of existing pandas dataframe wih extracted text data

        Args:
            path (str): path to directory containing text files
            
            label (int): class label of document
        Returns:
            dataframe (pd.DataFrame): dataframe with extracted text and class labels
    """
    #out of this list we will create a dataframe
    data = []
    #open directory
    folder = os.listdir(path)
    for i in folder:
        files = os.path.join(path, i)
        #extract text from each file in directory and append the values to the dataframe
        with open(files, 'r') as file:
            review_text = file.read()
            #add the value pairs review text and sentiment as dictionaries to list so that dictionary can be made into dataframe
            data.append({'sentiment': label,'review': review_text})
    #convert list to dataframe. Pandas can interpret the column row relationship based of key value pairs
    data = pd.DataFrame(data)
    return data

df_neg = get_docs(neg_path, 0)
df_pos = get_docs(pos_path, 1)
#need to union the data together 

#shuffle data so that the model does not interpret a pattern based on position of sentiments (all the negative reviews being listed first and the postive ones last)
df = pd.concat([df_neg, df_pos], axis = 0).sample(frac = 1).reset_index(drop = True)
df.head()


,sentiment,review
0,0,my inner flag was at half-mast last year when ...
1,1,""" when you get out of jail , you can kill him..."
2,1,playwright tom stoppard and screenwriter marc ...
3,1,"capsule : trippy , hyperspeed action machine f..."
4,0,the most absurd remake of 1998 ? \nit's a toss...


There are punctiation marks and in the fifth row there is a newline character ('\n') for every time the sentence continues on to the next line of the text file. All of the punctuation marks and newline characters have to be removed from the text so that they are not intepreted as a letter or an individual word during modeling. Stop words like 'to' and 'and' need to be removed because they tend to have higher term frequencies than other words. Additonally all the words will be turned to lower case so that a single word will not be interpreted twice.

Cleaning the data will also require for lemmatization or stemming of the text. Lemmatization and stemming are done using nltk and their goal is to reduce the frequency of words with the same meaning. 

**Stemming**

-Reduce the word to its root form. For example, 'bought' and 'buying' are reduced to 'buy'

**Lemmatization**

-Reduces the word to its definitional form. For example, 'am' and 'is' are reduced to 'be'



**Exploratory Data Analysis**

Preprocessing Data 

**Feature Extraction**

**Naïve Bayes Classifier Training**

**Model Testing**

**Conclusion**